# Entrenament de Models - Classificació de Pingüins Palmer

Aquest notebook entrena 4 models diferents per classificar les espècies de pingüins:
1. Regressió Logística
2. SVM (Support Vector Machine)
3. Arbres de Decisió (Decision Trees)
4. KNN (k-Nearest Neighbours)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

# Carregar el dataset net
df = pd.read_csv('../datasets/penguins_clean.csv')
print(f"Dataset carregat amb {len(df)} files")
df.head()

Dataset carregat amb 333 files


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,torgersen,36.7,19.3,193.0,3450.0,female
4,Adelie,torgersen,39.3,20.6,190.0,3650.0,male


## 1. Preparació de les dades

In [3]:
# Definir variables categòriques i numèriques
categorical = ['island', 'sex']
numerical = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

print("Variables categòriques:", categorical)
print("Variables numèriques:", numerical)

Variables categòriques: ['island', 'sex']
Variables numèriques: ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']


In [4]:
# Codificar la variable objectiu (species)
le = LabelEncoder()
df['species_encoded'] = le.fit_transform(df['species'])

print("\nCodificació d'espècies:")
for i, species in enumerate(le.classes_):
    print(f"{i}: {species}")


Codificació d'espècies:
0: Adelie
1: Chinstrap
2: Gentoo


In [5]:
# Dividir en entrenament (80%) i prova (20%)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['species_encoded'])

print(f"\nDades d'entrenament: {len(df_train)} ({len(df_train)/len(df)*100:.1f}%)")
print(f"Dades de prova: {len(df_test)} ({len(df_test)/len(df)*100:.1f}%)")

# Separar X i y
y_train = df_train['species_encoded'].values
y_test = df_test['species_encoded'].values

print(f"\nDistribució d'espècies en entrenament:")
print(pd.Series(y_train).value_counts().sort_index())
print(f"\nDistribució d'espècies en prova:")
print(pd.Series(y_test).value_counts().sort_index())


Dades d'entrenament: 266 (79.9%)
Dades de prova: 67 (20.1%)

Distribució d'espècies en entrenament:
0    117
1     54
2     95
Name: count, dtype: int64

Distribució d'espècies en prova:
0    29
1    14
2    24
Name: count, dtype: int64


## 2. Enginyeria de propietats (Feature Engineering)

In [6]:
# Convertir a diccionaris per DictVectorizer
train_dict = df_train[categorical + numerical].to_dict(orient='records')
test_dict = df_test[categorical + numerical].to_dict(orient='records')

# Codificació one-hot amb DictVectorizer
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)
X_test = dv.transform(test_dict)

print(f"\nForma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"\nCaracterístiques generades:")
print(dv.get_feature_names_out())


Forma de X_train: (266, 9)
Forma de X_test: (67, 9)

Característiques generades:
['bill_depth_mm' 'bill_length_mm' 'body_mass_g' 'flipper_length_mm'
 'island=biscoe' 'island=dream' 'island=torgersen' 'sex=female' 'sex=male']


In [7]:
# Normalitzar les variables numèriques
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nEstadístiques de X_train_scaled:")
print(f"Mitjana: {X_train_scaled.mean(axis=0).mean():.4f}")
print(f"Desviació estàndard: {X_train_scaled.std(axis=0).mean():.4f}")


Estadístiques de X_train_scaled:
Mitjana: -0.0000
Desviació estàndard: 1.0000


## 3. Entrenament dels Models

In [10]:
# Funció per avaluar models
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n{'='*60}")
    print(f"MODEL: {model_name}")
    print(f"{'='*60}")
    print(f"\nAccuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"\nInforme de classificació:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    print(f"\nMatriu de confusió:")
    print(confusion_matrix(y_test, y_pred))
    
    return accuracy

### 3.1. Regressió Logística

In [11]:
# Entrenar Regressió Logística
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

# Avaluar
lr_accuracy = evaluate_model(lr_model, X_test_scaled, y_test, "Regressió Logística")


MODEL: Regressió Logística

Accuracy: 0.9851 (98.51%)

Informe de classificació:
              precision    recall  f1-score   support

      Adelie       1.00      0.97      0.98        29
   Chinstrap       0.93      1.00      0.97        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           0.99        67
   macro avg       0.98      0.99      0.98        67
weighted avg       0.99      0.99      0.99        67


Matriu de confusió:
[[28  1  0]
 [ 0 14  0]
 [ 0  0 24]]


### 3.2. SVM (Support Vector Machine)

In [12]:
# Entrenar SVM
svm_model = SVC(kernel='rbf', random_state=42, probability=True)
svm_model.fit(X_train_scaled, y_train)

# Avaluar
svm_accuracy = evaluate_model(svm_model, X_test_scaled, y_test, "SVM")


MODEL: SVM

Accuracy: 0.9851 (98.51%)

Informe de classificació:
              precision    recall  f1-score   support

      Adelie       1.00      0.97      0.98        29
   Chinstrap       0.93      1.00      0.97        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           0.99        67
   macro avg       0.98      0.99      0.98        67
weighted avg       0.99      0.99      0.99        67


Matriu de confusió:
[[28  1  0]
 [ 0 14  0]
 [ 0  0 24]]


### 3.3. Arbres de Decisió (Decision Trees)

In [13]:
# Entrenar Arbres de Decisió
dt_model = DecisionTreeClassifier(random_state=42, max_depth=5)
dt_model.fit(X_train_scaled, y_train)

# Avaluar
dt_accuracy = evaluate_model(dt_model, X_test_scaled, y_test, "Arbres de Decisió")


MODEL: Arbres de Decisió

Accuracy: 0.9254 (92.54%)

Informe de classificació:
              precision    recall  f1-score   support

      Adelie       0.93      0.90      0.91        29
   Chinstrap       0.82      1.00      0.90        14
      Gentoo       1.00      0.92      0.96        24

    accuracy                           0.93        67
   macro avg       0.92      0.94      0.92        67
weighted avg       0.93      0.93      0.93        67


Matriu de confusió:
[[26  3  0]
 [ 0 14  0]
 [ 2  0 22]]


### 3.4. KNN (k-Nearest Neighbours)

In [14]:
# Entrenar KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Avaluar
knn_accuracy = evaluate_model(knn_model, X_test_scaled, y_test, "KNN")


MODEL: KNN

Accuracy: 0.9851 (98.51%)

Informe de classificació:
              precision    recall  f1-score   support

      Adelie       1.00      0.97      0.98        29
   Chinstrap       0.93      1.00      0.97        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           0.99        67
   macro avg       0.98      0.99      0.98        67
weighted avg       0.99      0.99      0.99        67


Matriu de confusió:
[[28  1  0]
 [ 0 14  0]
 [ 0  0 24]]


## 4. Comparació de Models

In [15]:
# Comparar resultats
results = pd.DataFrame({
    'Model': ['Regressió Logística', 'SVM', 'Arbres de Decisió', 'KNN'],
    'Accuracy': [lr_accuracy, svm_accuracy, dt_accuracy, knn_accuracy]
})

results = results.sort_values('Accuracy', ascending=False).reset_index(drop=True)
print("\n" + "="*60)
print("RESUM DE RESULTATS")
print("="*60)
print(results.to_string(index=False))
print(f"\nMillor model: {results.iloc[0]['Model']} amb {results.iloc[0]['Accuracy']*100:.2f}% d'accuracy")


RESUM DE RESULTATS
              Model  Accuracy
Regressió Logística  0.985075
                SVM  0.985075
                KNN  0.985075
  Arbres de Decisió  0.925373

Millor model: Regressió Logística amb 98.51% d'accuracy


## 5. Serialització dels Models

In [16]:
# Guardar tots els models
models = {
    'logistic_regression': (dv, scaler, le, lr_model),
    'svm': (dv, scaler, le, svm_model),
    'decision_tree': (dv, scaler, le, dt_model),
    'knn': (dv, scaler, le, knn_model)
}

for model_name, model_tuple in models.items():
    filename = f'../models/{model_name}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model_tuple, f)
    print(f"Model guardat: {filename}")

print("\n✓ Tots els models han estat serialitzats correctament!")

Model guardat: ../models/logistic_regression_model.pkl
Model guardat: ../models/svm_model.pkl
Model guardat: ../models/decision_tree_model.pkl
Model guardat: ../models/knn_model.pkl

✓ Tots els models han estat serialitzats correctament!
